In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [19]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL
from typing import List, Dict, Annotated
from langchain.tools import tool
from bs4 import BeautifulSoup
import requests
import re

from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

from langchain_teddynote.messages import AgentCallbacks
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [5]:
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [6]:
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    
    response = requests.get(news_url)           # HTTP GET 요청 보내기

    if response.status_code == 200:             # 요청 성공
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)      # \n 이 2번이상 반복되는 것을 찾아서 \n으로 바꾼다.
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [7]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [8]:
tools = [search_news, naver_news_crawl, python_repl_tool]

In [11]:
# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant."             
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [15]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

In [20]:
# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    print(f"Tool: {tool.get('tool')}")  # 사용된 도구의 이름을 출력합니다.
    print("<<<<<<< 도구 호출 >>>>>>")


# 관찰 결과를 출력하는 콜백 함수입니다.
def observation_callback(observation) -> None:
    print("<<<<<<< 관찰 내용 >>>>>>")
    print(
        f"Observation: {observation.get('observation')[0]}"
    )  # 관찰 내용을 출력합니다.
    print("<<<<<<< 관찰 내용 >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)  # 최종 답변을 출력합니다.
    print("<<<<<<< 최종 답변 >>>>>>")


# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정합니다.
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,
    observation_callback=observation_callback,
    result_callback=result_callback,
)

# AgentStreamParser 객체를 생성하여 에이전트의 실행 과정을 파싱합니다.
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [21]:
result = agent_executor.stream({"input": "AI 투자관련 뉴스를 검색해 주세요."})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: search_news
<<<<<<< 도구 호출 >>>>>>

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE9HMzNkMXZaZ0k2ekVETmtLNVI2bVVrQlBkbVE3Ry1IenhrWkR1d2pCeHk1ZnNIVjVSVWx4VDRTdjZxcWtsdlhLYTJ4aFVzQzkyNHhsa2pB?oc=5', 'content': '"투자·인재·제도 열악한데 AI 3강?"…韓, 마이너리그 전락 \'우려\' - ZD넷 코리아'}, {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE5Xd2VwektIS1FFd29uU2hHNzNUQnQ4QVlhekR0YWlSZmk1c2lialRURy1Rd2xFT2lnM3ZEeTdvUVBRY2ltd204NXFpTEM4UE1Hb1NjalpB?oc=5', 'content': "트웰브랩스, '430억' 투자 유치…영상 AI 혁신 '가속화' - ZD넷 코리아"}, {'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTFA4T3ZXMmM5VUVTSUxsYzhPaktQMWtzcE5VUHlUaUNMV0FpSmVacTRFWjd0TlhYTXVpdWdZNExFcUxWZTZMRmk5RDh0TXVkeVhfcjdXT3JnZzRNWXNkZFRrTkNTbDc3Skx3SHBacW9KeENvQQ?oc=5', 'content': '오픈AI가 픽한 유니콘…‘스픽’ 1100억 투자 유치 - 마켓인'}, {'url': 'https://news.google.com/rss/articles/CBMicEFVX3lxTE0wenpQNDRGXzM3clotcG1jSU91YTNlYThLSVJxN2RzME5ldlhsdFBvQmZ

In [27]:
store = {}

# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:                    # session_id 가 store에 없는 경우
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        
    return store[session_ids]                       # 해당 세션 ID에 대한 세션 기록 반환

In [28]:
# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,    
    get_session_history,                        # 대화 session_id    
    input_messages_key="input",                 # 프롬프트의 질문이 입력되는 key: "input"    
    history_messages_key="chat_history",        # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

In [29]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "안녕? 내 이름은 이인환이야!"},    
    config={
        "configurable": {"session_id": "abc123"}    # session_id 설정
    },      
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
안녕하세요, 이인환님! 어떻게 도와드릴까요?

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
안녕하세요, 이인환님! 어떻게 도와드릴까요?
<<<<<<< 최종 답변 >>>>>>


In [30]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "내 이름이 뭐라고 했어?"},    
    config={
        "configurable": {"session_id": "abc123"}    # session_id 설정
    },      
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
이인환님이라고 하셨습니다! 맞나요?

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
이인환님이라고 하셨습니다! 맞나요?
<<<<<<< 최종 답변 >>>>>>


In [31]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "내 이메일 주소는 loveinlove02@naver.com 이야. 회사 이름은 소프트웨어놀이터."},    
    config={
        "configurable": {"session_id": "abc123"}    # session_id 설정
    },      
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
알겠습니다, 이인환님! 이메일 주소와 회사 이름을 기억하겠습니다. 어떤 도움을 드릴까요?

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
알겠습니다, 이인환님! 이메일 주소와 회사 이름을 기억하겠습니다. 어떤 도움을 드릴까요?
<<<<<<< 최종 답변 >>>>>>


In [32]:
response = agent_with_chat_history.stream(
    {
        "input": "최신 뉴스 5개를 검색해서 이메일의 본문으로 작성해줘. "
        "수신인에는 `윤정 팀장님` 그리고, 발신인에는 내 인적정보를 적어줘."
        "정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘."
    },
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: search_news
<<<<<<< 도구 호출 >>>>>>

Invoking: `search_news` with `{'query': '최신 뉴스'}`


[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFB6b3FPREFINURLSy1pVUZDbXB4X3JwaW1fVGNCRks1RklJMDFNTWxCbURTbGMwU05kWVRRLU9xNnVFUkNPVFNLaUlTOThQaDRoQzlKQWtsQnVETnFqbEhGc2hIQnU2RERoS3c?oc=5', 'content': '인사이드 구글 - The Keyword'}, {'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTE5ZMFp6LVRhUG9hamNMVkZTMnM5ajdWbjFMWkdMSk1oZDZVODFKc1hKT3RyUmpRakp4SnlJZ0FkbEJCWFdQaFZjcFozbFZiMlJwMnZKUDFUb3hSWkhxQU54NnJpZGlrdTFl0gFsQVVfeXFMUDZqbks3RG1Pcnp5cy1iY3BReTRpdnp6UjNBQTh4NVFMZWtQMUN1Z0E5ZDllcGo5MmtKY3VyU0JSYlc1NVRUdzdMdmZYNVBFTDVtNmlVMGloUm1BdVRtX29kNk5NVXpJSV9hSTV4?oc=5', 'content': '세탁기 안 들어가는 아파트 세탁실···하자분쟁, 어떻게 해결했나 - 대한전문건설신문'}, {'url': 'https://news.google.com/rss/articles/CBMiV0FVX3lxTE45MnItRDB5X3poVlBIbEtyTVhoZE9fYVUxdFVfR2NMNHNCZW5QRnVWbXNkbmxCMkxCQlo5QW5wRVVTSFk5UE1Rem1lQ2JkS0pteVpWMS1rUQ?oc=5', 'content': '[주요 뉴스] 도지코인, 노드 최신 버